

You need to provide your lingq api key when prompted which you can get from here:

https://www.lingq.com/en/accounts/apikey/


To run the script select "Run all" from the "Runtime" menu above.

# Initial set up


In [ ]:
#Using Google colab secrets to store lingq api key.
from google.colab import userdata

import urllib.parse

import requests

import json


# Helper Functions

In [ ]:
def get_json_response (url):
  lingq_api_key=userdata.get('LINGQ_API_KEY');
  headers = {
    'Authorization': f'Token {lingq_api_key}',
    'Content-Type': 'application/json'
  }
  response = requests.get(f'{url}?page_size=1000', headers=headers)
  return response.json()

def print_json (json_parsed):
  print (json.dumps(json_parsed, indent = 4))


In [ ]:
import spacy
from ipywidgets import widgets
from IPython.display import display, HTML
#@title Choose a language model
model = "pl_core_news_lg" #@param ["fi_core_news_lg", "de_core_news_lg", "en_core_web_lg", "nl_core_news_lg", "sv_core_news_lg", "da_core_news_lg", "pl_core_news_lg", "ru_core_news_lg", "uk_core_news_lg"]
!python -m spacy download {model}

spacy.prefer_gpu()

nlp = spacy.load(model)

# Function to process text and return HTML with color-coded cases and tooltips
def process_and_display_paragraph(paragraph):
    doc = nlp(paragraph)
    highlighted_text = ""
    for token in doc:
        # Retrieve morphological information
        case = token.morph.get("Case")
        morph = " ".join(f'{token.morph}'.split("|"))
        # Define the tooltip text
        tooltip_text = f"{token.lemma_} {token.pos_} ({morph})"

        # Assign color based on grammatical case
        color = "black"  # Default color
        if case:
            if "Nom" in case:
                color = "blue"
            elif "Gen" in case:
                color = "green"
            elif "Dat" in case:
                color = "red"
            elif "Acc" in case:
                color = "purple"
            elif "Ins" in case:
                color = "orange"
            elif "Loc" in case:
                color = "brown"
            elif "Voc" in case:
                color = "pink"

        # Append the token span with style and tooltip
        highlighted_text += f'<span style="color: {color};" title="{tooltip_text}">{token.text}</span> '

    display(HTML(f"<p>{highlighted_text}</p>"))


In [ ]:
course = get_json_response(f'https://www.lingq.com/api/v2/pl/collections/289027')

for lesson in course['lessons']:
  lesson_details = get_json_response(lesson['url'])
  paragraphs = get_json_response(lesson['url'] + 'paragraphs/')
  display(HTML(f"<h2>{lesson_details['title']}</h2>"))
  for paragraph in paragraphs:
    joined_sentences = ' '.join(sentence['cleanText'] for sentence in paragraph['sentences'])
    process_and_display_paragraph(joined_sentences)


